In [6]:
import pandas as pd
import numpy as np

df = pd.read_csv('test_file.csv')


In [7]:
df.head(100)

,metric_id,display_names,operating_system_type,Hierarchy_ID,Parent_ID,8/29/2024,8/30/2024,8/31/2024,9/1/2024,9/2/2024,9/3/2024,9/4/2024
0,139,Devices Page Visits,Apple iOS,Apple iOS_SH_1,NaN,3444,3530,3271,3036,3104,3635,3879
1,140,Started Troubleshooting (Solution Center),Apple iOS,Apple iOS_SH_1.1,Apple iOS_SH_1,80,104,112,86,84,90,105
2,141,Saw a Result,Apple iOS,Apple iOS_SH_1.1.1,Apple iOS_SH_1.1,76,100,110,85,82,85,103
3,142,1st common reason (Unknown),Apple iOS,Apple iOS_SH_1.1.1.1,Apple iOS_SH_1.1.1,60,84,89,58,64,63,76
4,143,2nd common reason (Disconnected),Apple iOS,Apple iOS_SH_1.1.1.2,Apple iOS_SH_1.1.1,11,10,13,16,13,13,17
5,144,3rd common reason (Incomplete Install),Apple iOS,Apple iOS_SH_1.1.1.3,Apple iOS_SH_1.1.1,4,4,8,7,1,5,5
6,145,Did a Device Reboot,Apple iOS,Apple iOS_SH_1.2,Apple iOS_SH_1,32,43,59,36,41,35,43
7,146,Started Proactive and Preventive (P&P),Apple iOS,Apple iOS_SH_1.3,Apple iOS_SH_1,2045,2036,1979,1868,1908,2161,2326
8,147,Started P&P troubleshooting,Apple iOS,Apple iOS_SH_1.3.1,Apple iOS_SH_1.3,1130,1126,1136,1022,1081,1197,1344
9,148,Smarthelp Reset Complete,Apple iOS,Apple iOS_SH_1.3.2,Apple iOS_SH_1.3,400,398,385,360,346,371,371


In [8]:
date_columns = ['8/29/2024', '8/30/2024', '8/31/2024', '9/1/2024', '9/2/2024', '9/3/2024', '9/4/2024']

In [9]:
import pandas as pd
import numpy as np

def process_display_names1(df):
    """
    Generate new rows for each unique display_name based on given conditions.

    Args:
    - df (DataFrame): Input DataFrame containing the original data.

    Returns:
    - DataFrame: Updated DataFrame with newly calculated rows.
    """
    # Identify date columns
    date_columns = ['8/29/2024', '8/30/2024', '8/31/2024', '9/1/2024', '9/2/2024', '9/3/2024', '9/4/2024']
    
    new_rows = []  # Store new rows
    seq_num = 1  # Sequential ID for metric_id

    for display_name in df['display_names'].unique():
        # Filter rows for the current display_name
        display_df = df[df['display_names'] == display_name]

        # Case 1: If `parent_id_old` has null or empty values
        if display_df['Parent_ID'].isnull().any() or display_df['Parent_ID'].eq('').any():
            # Sum of `Apple iOS` and `Google Android` for `parent_id_old` being null
            ios_rows = display_df[display_df["operating_system_type"] == "Apple iOS"]
            android_rows = display_df[display_df["operating_system_type"] == "Google Android"]
            
            # Calculate numerator
            ios_sums = ios_rows[date_columns].replace(0, pd.NA).reset_index(drop=True)
            android_sums = android_rows[date_columns].replace(0, pd.NA).reset_index(drop=True)
            numerator = ios_sums + android_sums
            
            # Store numerator in new row
            new_row_values = numerator.sum(axis=0).fillna(0).to_dict()
        else:
            # Case 2: If `parent_id_old` is not null
            ios_rows = display_df[(display_df["operating_system_type"] == "Apple iOS") & 
                                  (display_df["Parent_ID"].notna()) & 
                                  (display_df["Parent_ID"] != '')]
            android_rows = display_df[(display_df["operating_system_type"] == "Google Android") & 
                                      (display_df["Parent_ID"].notna()) & 
                                      (display_df["Parent_ID"] != '')]
            
            # Calculate numerator
            ios_sums = ios_rows[date_columns].replace(0, pd.NA).reset_index(drop=True)
            android_sums = android_rows[date_columns].replace(0, pd.NA).reset_index(drop=True)
            numerator = ios_sums + android_sums
            
            # Filter rows where `parent_id_old` matches `hierarchy_id`
            filtered_data = df[df['Hierarchy_ID'].isin(display_df['Parent_ID'])]
            ios_data = filtered_data[filtered_data['operating_system_type'] == "Apple iOS"]
            android_data = filtered_data[filtered_data['operating_system_type'] == "Google Android"]

            # Calculate denominator
            ios_sums_denom = ios_data[date_columns].reset_index(drop=True)
            android_sums_denom = android_data[date_columns].reset_index(drop=True)
            denominator = ios_sums_denom + android_sums_denom

            # Calculate result as percentage
            result = (numerator / denominator) * 100 if not denominator.empty else pd.DataFrame(0, columns=date_columns, index=numerator.index)
            new_row_values = result.sum(axis=0).fillna(0).to_dict()

        # Create the new row for the current display_name
        new_row = {
            'display_names': display_name,
            'operating_system_type': 'Both',
            'metric_id': f'Both_ID_{seq_num}',
            'Hierarchy_ID': np.nan,
            'Parent_id_old': np.nan,
            'Parent_id': np.nan
        }
        # Add calculated values to the new row
        new_row.update(new_row_values)
        new_rows.append(new_row)
        seq_num += 1

    # Append new rows to the original DataFrame
    new_rows_df = pd.DataFrame(new_rows)
    result_df = pd.concat([df, new_rows_df], ignore_index=True)

    # Reset index
    result_df.reset_index(drop=True, inplace=True)
    return new_rows_df

In [10]:
# Assuming 'df' is the DataFrame containing your data
updated_ddf = process_display_names1(df)

# View the result
print(updated_ddf)


                                display_names operating_system_type  \
0                         Devices Page Visits                  Both   
1   Started Troubleshooting (Solution Center)                  Both   
2                                Saw a Result                  Both   
3                 1st common reason (Unknown)                  Both   
4            2nd common reason (Disconnected)                  Both   
5      3rd common reason (Incomplete Install)                  Both   
6                         Did a Device Reboot                  Both   
7      Started Proactive and Preventive (P&P)                  Both   
8                 Started P&P troubleshooting                  Both   
9                    Smarthelp Reset Complete                  Both   
10                              Schedule Tech                  Both   
11                         Connect with Agent                  Both   
12         Smarthelp_P&P Interacted with Chat                  Both   
13    